In [1]:
import theano
import numpy as np
from keras.layers import GlobalAveragePooling1D

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using Theano backend.


In [2]:
from __future__ import print_function
import theano
import numpy as np
import nltk
import itertools
from nltk.tokenize import TweetTokenizer
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras.optimizers import Adam
import csv
from keras import backend as K
from keras.layers import Flatten
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D

In [3]:
import nltk
import itertools
from nltk.tokenize import TweetTokenizer
import pandas as pd

In [4]:
def arrangeTweet(tweet):
    return ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))

In [5]:
arrangeTweet('LOOOOOVEEEE ')

'LOVE '

In [6]:
vocabulary_size = 10000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

oldfile=open("train_pos.txt",'rb')
sentences=[]
i=0
for line in oldfile:
    if i%50000==0:
        print(i)
    i=i+1
    
    line = line.decode('utf8')
    line=arrangeTweet(line)
    #if not (line in sentences):
    sentences.append(line)

0
50000


In [7]:
len(sentences)

100000

In [8]:
oldfile=open("train_neg.txt",'rb')
i=0
for line in oldfile:
    if i%50000==0:
        print(i)
    i=i+1
    
    line = line.decode('utf8')
    line=arrangeTweet(line)
    #if not (line in sentences):
    sentences.append(line)

0
50000


In [1]:
len(sentences)

NameError: name 'sentences' is not defined

In [10]:
#indexes=np.arange(0,181255)
indexes=np.arange(0,200000)

np.random.shuffle(indexes)

In [11]:
indexes

array([ 64404, 175630, 180009, ...,   9448, 188605,   3223])

In [12]:
y=[0]*100000
y2=[1]*100000
y=y+y2
y=np.array(y)

In [13]:
y=y[indexes]

In [14]:
y

array([0, 1, 1, ..., 0, 1, 0])

In [15]:
len(sentences)

200000

In [16]:
tweetTok=TweetTokenizer()

In [17]:
tokenized_sentences = [tweetTok.tokenize(sent) for sent in sentences]

In [18]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))

Found 104326 unique words tokens.


In [19]:
vocab = word_freq.most_common(20000)

In [20]:
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [21]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]


X = np.asarray([[word_to_index[w]+1 for w in sent] for sent in tokenized_sentences])

In [ ]:
testfile=open("test_data.txt",'rb')
i=0
sentences=[]
for line in testfile:
    if i%5000==0:
        print(i)
    i=i+1
    line=line.decode('utf8')
    sentences.append((line[line.find(',')+1:]))

In [ ]:
tokenized_sentences = [tweetTok.tokenize(sent) for sent in sentences]

In [ ]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]


X_test = np.asarray([[word_to_index[w]+1 for w in sent] for sent in tokenized_sentences])

In [22]:
from collections import Counter

In [ ]:
test=set()
test.update((2,4))
test.update((2,3))
test

In [ ]:
ngram_set = []
for input_list in X:
        for i in range(2, 3):
            set_of_ngram = create_ngram_set(input_list, ngram_value=2)
            ngram_set+=set_of_ngram

In [ ]:
[x[0] for x in Counter(ngram_set).most_common()[0:5000]]

In [23]:
def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return list(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.
    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]
    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337], [1, 3, 7, 9, 2, 1337, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for i in range(len(new_list)-ngram_range+1):
            for ngram_value in range(2, ngram_range+1):
                ngram = tuple(new_list[i:i+ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences

In [24]:
max([len(x) for x in X])

64

In [26]:
ngram_range = 4
max_features = len(vocab)
maxlen = 64
batch_size = 200
embedding_dims = 50
nb_epoch = 3

print('Average train sequence length: {}'.format(np.mean(list(map(len, X)), dtype=int)))

Average train sequence length: 15


In [28]:
if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = []
    for input_list in X:
        for i in range(2, ngram_range+1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set+=set_of_ngram
    ngram_set=[x[0] for x in Counter(ngram_set).most_common()[0:10000]]

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k+start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting X_train and X_test with n-grams features
    X = add_ngram(X, token_indice, ngram_range)
    #X_test = add_ngram(X_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(np.mean(list(map(len, X)), dtype=int)))

Adding 4-gram features
Average train sequence length: 23


In [30]:
max([len(x) for x in X])

162

In [31]:
print('Pad sequences (samples x time)')
X = sequence.pad_sequences(X, maxlen=162, padding='pre')
#X_test = sequence.pad_sequences(X_test, maxlen=129, padding='pre')
print('X shape:', X.shape)

Pad sequences (samples x time)
X shape: (200000, 162)


In [32]:
maxlen=162

In [33]:
max_features=np.max(X)+1

In [34]:
max_features

40002

In [35]:
X=X[indexes]

In [ ]:
str="good -0.54403 0.60274 -0.14543 -0.023398 -0.13771 0.60137 2.192 0.20804 -0.51536 -0.23101 -0.80387 0.56901 -5.0234 0.26507 0.47891 -0.59854 0.56132 -1.0905 -0.52587 0.12506 -0.22624 0.24529 -0.45767 0.92619 0.022125"
np.array([float(c) for c in str.split()[1:]])
str.split()[0]

# PRETRAINED

In [ ]:
import codecs

In [ ]:
oldfile=open("glove.twitter.27B.200d.txt",'rb')
i=1
j=0
vocab_dim = 200
word_vectors={}
for i, line in enumerate(codecs.open("glove.twitter.27B.200d.txt", 'r', 'utf-8')):
    line = line.rstrip().split()
    if i%100000==0:
        print(i)
    #line=line.decode('utf8')
    if len(line) == vocab_dim + 1:
        i=i+1
        word_vectors[line[0]]=np.array([float(x) for x in line[1:]]).astype(np.float32)
    else:
        j += 1
print(i)
print(j)

In [ ]:
X.shape

In [ ]:
i=0
vocab_dim = 200 # dimensionality of your word vectors
n_symbols = len(word_to_index)+1 # adding 1 to account for 0th index (for masking)
embedding_weights = np.zeros((n_symbols,vocab_dim))
for word,index in word_to_index.items():
    if word in word_vectors:
        i+=1
        embedding_weights[index,:] = word_vectors[word]

In [ ]:
embedding_weights.shape

In [39]:

opt = Adam(lr=0.001)
embedding_dims=75
model = Sequential()
model.add(Embedding(max_features+1, embedding_dims,input_length=maxlen)) #weights=[embedding_weights]))
#model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(GlobalAveragePooling1D())

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y,batch_size=50,nb_epoch=10,validation_split=0.1)

Train on 180000 samples, validate on 20000 samples
Epoch 1/10
180000/180000 [==============================] - 189s - loss: 0.4781 - acc: 0.7691 - val_loss: 0.3980 - val_acc: 0.8180
Epoch 2/10
180000/180000 [==============================] - 224s - loss: 0.3654 - acc: 0.8381 - val_loss: 0.3724 - val_acc: 0.8325
Epoch 3/10
180000/180000 [==============================] - 239s - loss: 0.3388 - acc: 0.8531 - val_loss: 0.3676 - val_acc: 0.8322
Epoch 4/10
180000/180000 [==============================] - 246s - loss: 0.3241 - acc: 0.8609 - val_loss: 0.3690 - val_acc: 0.8326
Epoch 5/10
180000/180000 [==============================] - 254s - loss: 0.3143 - acc: 0.8657 - val_loss: 0.3681 - val_acc: 0.8358
Epoch 6/10
180000/180000 [==============================] - 258s - loss: 0.3070 - acc: 0.8687 - val_loss: 0.3722 - val_acc: 0.8347
Epoch 7/10
180000/180000 [==============================] - 267s - loss: 0.3016 - acc: 0.8713 - val_loss: 0.3784 - val_acc: 0.8329
Epoch 8/10
180000/180000 [======

In [ ]:
model.predict_proba(X_test)

In [ ]:
model = Sequential()
model.add(Embedding(max_features+1, embedding_dims,input_length=93))
model.add(Convolution1D(nb_filter=100, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_length=2))
#model.add(Flatten())
#model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y,batch_size=50,nb_epoch=4,validation_split=0.1)

In [38]:
opt = Adam(lr=0.001)
embedding_dims=100
model = Sequential()
model.add(Embedding(max_features+1, embedding_dims,input_length=maxlen)
#model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(GlobalAveragePooling1D())

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y,batch_size=100,nb_epoch=4,validation_split=0.1)

SyntaxError: invalid syntax (<ipython-input-38-a2f1c88c1873>, line 6)

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

In [ ]:
y_pred=1-2*y_pred

In [ ]:
import csv

In [ ]:
create_csv_submission(np.arange(1,10001),y_pred,'prediction444.csv')